In [1]:
# Clone the dataset
# Making use of ImageDataGenerator => create batches and do the augmentation
# Load the pretrained Model
# Pop Some layers, add new layers
# Train on new data
# Evaluate the model

In [2]:
!git clone https://github.com/Horea94/Fruit-Images-Dataset.git

Cloning into 'Fruit-Images-Dataset'...
remote: Enumerating objects: 385858, done.
remote: Counting objects: 100% (8693/8693), done.
remote: Compressing objects: 100% (8659/8659), done.
remote: Total 385858 (delta 37), reused 8688 (delta 34), pack-reused 377165
Receiving objects: 100% (385858/385858), 2.10 GiB | 22.67 MiB/s, done.
Resolving deltas: 100% (1197/1197), done.
Updating files: 100% (90503/90503), done.


In [3]:
!ls Fruit-Images-Dataset/Test

'Apple Braeburn'       'Grape Blue'	     'Pear Monster'
'Apple Crimson Snow'   'Grapefruit Pink'     'Pear Red'
'Apple Golden 1'       'Grapefruit White'    'Pear Stone'
'Apple Golden 2'       'Grape Pink'	     'Pear Williams'
'Apple Golden 3'       'Grape White'	      Pepino
'Apple Granny Smith'   'Grape White 2'	     'Pepper Green'
'Apple Pink Lady'      'Grape White 3'	     'Pepper Orange'
'Apple Red 1'	       'Grape White 4'	     'Pepper Red'
'Apple Red 2'	        Guava		     'Pepper Yellow'
'Apple Red 3'	        Hazelnut	      Physalis
'Apple Red Delicious'   Huckleberry	     'Physalis with Husk'
'Apple Red Yellow 1'    Kaki		      Pineapple
'Apple Red Yellow 2'    Kiwi		     'Pineapple Mini'
 Apricot	        Kohlrabi	     'Pitahaya Red'
 Avocado	        Kumquats	      Plum
'Avocado ripe'	        Lemon		     'Plum 2'
 Banana		       'Lemon Meyer'	     'Plum 3'
'Banana Lady Finger'    Limes		      Pomegranate
'Banana Red'	        Lychee		     'Pomelo Sweetie'
 Beetroot	        Mand

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, Flatten, Convolution2D

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
# Instantiating the ImageDataGenerator (preprocessing_function,rotation_range,width_shift_range,height_range,horizontal_rotation)
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,horizontal_flip=True,width_shift_range=0.2,height_shift_range=0.2,zoom_range=0.2)
test_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)

In [7]:
# Parsing the data in the generator, dataset should be such processed that each class should be in seperate diractory
train_generator = train_datagen.flow_from_directory(directory="Fruit-Images-Dataset/Training",batch_size=64,class_mode='categorical')

Found 67692 images belonging to 131 classes.


In [8]:
test_generator=test_datagen.flow_from_directory(directory="Fruit-Images-Dataset/Test",batch_size=64,class_mode='categorical')

Found 22688 images belonging to 131 classes.


In [9]:
base_model = MobileNet(weights='imagenet', include_top=False)

# Add custom layers on top of the base_model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)

# Output layer with the number of classes in your dataset
num_classes = len(train_generator.class_indices)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the custom model by specifying the input and output layers
model = Model(inputs=base_model.input, outputs=predictions)

17225924/17225924 [==============================] - 1s 0us/step


In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv1 (Conv2D)              (None, None, None, 32)    864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, None, None, 32)   128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, None, None, 32)    0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)   288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, None, None, 32)   128       
 ation)                                                      

In [11]:
# Optional: Freeze the layers of MobileNet so that they are not re-trained during the transfer learning process
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with an optimizer, loss, and metrics
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with your data
epochs = 3  # You can adjust the number of epochs based on the dataset size and convergence
model.fit(
    train_generator,
    epochs=epochs,
)


Epoch 1/3
1058/1058 [==============================] - 1145s 1s/step - loss: 0.6898 - accuracy: 0.8283
Epoch 2/3
1058/1058 [==============================] - 1112s 1s/step - loss: 0.1024 - accuracy: 0.9697
Epoch 3/3
1058/1058 [==============================] - 1107s 1s/step - loss: 0.0769 - accuracy: 0.9754


In [13]:
kevaluation = model.evaluate(test_generator)

355/355 [==============================] - 65s 182ms/step - loss: 0.0931 - accuracy: 0.9730


NameError: ignored

In [14]:
# The evaluation result will be a list containing the loss and metrics values
loss = kevaluation[0]
accuracy = kevaluation[1]

print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

Test Loss: 0.0931, Test Accuracy: 0.9730
